# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage.

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [ ]:
# Use as many python cells as you wish to write your code

In [ ]:
import pandas as pd

def merge_all_data(health_path, supplement_path, experiments_path, profiles_path):
    # -------------------------------------------------------
    # 1. EXTRACT: Load all datasets
    # -------------------------------------------------------
    df_health = pd.read_csv(health_path)
    df_supplements = pd.read_csv(supplement_path)
    df_experiments = pd.read_csv(experiments_path)
    df_profiles = pd.read_csv(profiles_path)

    # -------------------------------------------------------
    # 2. TRANSFORM: Fix Sleep Hours (CRITICAL FIX)
    # -------------------------------------------------------
    # The data has 'h' or 'H' at the end (e.g., "8.8h").
    # We must remove 'h' and convert to float.
    # str.replace with case=False handles both 'h' and 'H'
    df_health['sleep_hours'] = df_health['sleep_hours'].str.replace('h', '', case=False).astype(float)

    # -------------------------------------------------------
    # 3. TRANSFORM: Process User Profiles (Age Grouping)
    # -------------------------------------------------------
    def get_age_group(age):
        if pd.isna(age):
            return 'Unknown'
        try:
            age_int = int(age)
        except (ValueError, TypeError):
            return 'Unknown'

        if age_int < 18: return 'Under 18'
        elif age_int <= 25: return '18-25'
        elif age_int <= 35: return '26-35'
        elif age_int <= 45: return '36-45'
        elif age_int <= 55: return '46-55'
        elif age_int <= 65: return '56-65'
        else: return 'Over 65'

    df_profiles['user_age_group'] = df_profiles['age'].apply(get_age_group)

    # Clean text data: Remove whitespace from email and age_group
    df_profiles['email'] = df_profiles['email'].str.strip()
    df_profiles['user_age_group'] = df_profiles['user_age_group'].str.strip()
    df_profiles = df_profiles[['user_id', 'email', 'user_age_group']]

    # -------------------------------------------------------
    # 4. TRANSFORM: Process Experiments
    # -------------------------------------------------------
    # Rename 'name' to 'experiment_name'
    df_experiments = df_experiments.rename(columns={'name': 'experiment_name'})
    # Clean text: Strip whitespace
    df_experiments['experiment_name'] = df_experiments['experiment_name'].str.strip()

    # -------------------------------------------------------
    # 5. TRANSFORM: Process Supplement Usage
    # -------------------------------------------------------
    # Ensure dosage is numeric
    df_supplements['dosage'] = pd.to_numeric(df_supplements['dosage'], errors='coerce')

    # Function to convert dosage to grams
    def convert_dosage(row):
        if pd.isna(row['dosage']):
            return None
        # Check unit explicitly
        # Based on your inspection, unit is 'mg'
        unit = str(row['dosage_unit']).lower().strip()
        if 'mg' in unit:
            return row['dosage'] / 1000.0
        else:
            return row['dosage']

    df_supplements['dosage_grams'] = df_supplements.apply(convert_dosage, axis=1)

    # Ensure is_placebo is boolean
    if df_supplements['is_placebo'].dtype == object:
        bool_map = {'true': True, 'false': False, '1': True, '0': False}
        df_supplements['is_placebo'] = df_supplements['is_placebo'].str.lower().map(bool_map)

    # Clean text: Strip supplement names
    df_supplements['supplement_name'] = df_supplements['supplement_name'].str.strip()

    # Merge supplements with experiments
    df_supplements_merged = pd.merge(df_supplements, df_experiments, on='experiment_id', how='left')
    df_supplements_final = df_supplements_merged[[
        'user_id', 'date', 'supplement_name', 'dosage_grams', 'is_placebo', 'experiment_name'
    ]]

    # -------------------------------------------------------
    # 6. LOAD (Merge): Combine all data
    # -------------------------------------------------------
    df_merged = pd.merge(df_health, df_profiles, on='user_id', how='left')
    df_final = pd.merge(df_merged, df_supplements_final, on=['user_id', 'date'], how='left')

    # -------------------------------------------------------
    # 7. FINAL CLEANUP
    # -------------------------------------------------------
    # Handle "No intake"
    df_final['supplement_name'] = df_final['supplement_name'].fillna('No intake')

    # Convert date to datetime object
    df_final['date'] = pd.to_datetime(df_final['date'])

    # Define required columns
    required_columns = [
        'user_id',
        'date',
        'email',
        'user_age_group',
        'experiment_name',
        'supplement_name',
        'dosage_grams',
        'is_placebo',
        'average_heart_rate',
        'average_glucose',
        'sleep_hours',
        'activity_level'
    ]

    return df_final[required_columns]


In [ ]:
user_health_path='user_health_data.csv'
supplement_usage_path='supplement_usage.csv'
experiments_path='experiments.csv'
user_profiles_path='user_profiles.csv'
merge_all_data(
    user_health_path,
    supplement_usage_path,
    experiments_path,
    user_profiles_path)